# 🔐 Field-Level Encryption & decryption in PySpark

AES (Advanced Encryption Standard) and RSA (Rivest-Shamir-Adleman) are two widely used encryption algorithms, but they serve different purposes:
- AES is a symmetric encryption algorithm, meaning it uses the same key for both encryption and decryption. It is known for its speed and efficiency, making it ideal for encrypting large amounts of data. AES supports key sizes of 128, 192, and 256 bits, ensuring strong security.
- RSA is an asymmetric encryption algorithm, meaning it uses a pair of keys: a public key for encryption and a private key for decryption. RSA is commonly used for secure data transmission, digital signatures, and key exchanges. It relies on the mathematical difficulty of factoring large prime numbers, making it highly secure but slower than AES for bulk data encryption.
AES is often used for encrypting stored data, while RSA is typically used for secure communications and authentication. Sometimes, both are used together in hybrid encryption systems to balance security and efficiency. Let me know if you’d like more details!


This chapter demonstrates how to perform **encryption and decryption** of sensitive fields in a PySpark DataFrame using two Python libraries:

1. `cryptography`
2. `pycryptodome`

Both are useful in Microsoft Fabric Spark environments for securing personal or confidential data.


# 🔐 AES Field-Level Encryption in PySpark


AES supports several modes of operation, each designed for different use cases and security properties. Here are the common AES modes available in most cryptography libraries including pycryptodome and cryptography:

| Mode                            | Description                                                       | Use Case & Features                        |
| ------------------------------- | ----------------------------------------------------------------- | ------------------------------------------ |
| **ECB** (Electronic Codebook)   | Simplest mode, encrypts each block independently                  | Not recommended — patterns leak, insecure  |
| **CBC** (Cipher Block Chaining) | Each block XOR’d with previous ciphertext block before encryption | Common, secure if IV random & unique       |
| **CFB** (Cipher Feedback)       | Converts block cipher into stream cipher mode                     | Good for streaming data                    |
| **OFB** (Output Feedback)       | Similar to CFB, makes block cipher into stream cipher             | Resistant to error propagation             |
| **CTR** (Counter)               | Converts block cipher into stream cipher using counter values     | Highly parallelizable, popular in practice |
| **GCM** (Galois/Counter Mode)   | Authenticated encryption providing confidentiality and integrity  | Preferred mode for secure communications   |
| **EAX**                         | Authenticated encryption mode combining CTR + CBC-MAC             | Alternative authenticated encryption       |
| **CCM**                         | Authenticated encryption combining CTR + CBC-MAC                  | Used in some wireless protocols            |


# 🔐 AES(CBC) Field-Level Encryption in PySpark

## 📊 Sample Data

In [9]:
# Create sample Spark DataFrame with sensitive data
df = spark.createDataFrame([
    ("RamReddy Myla", "123-45-6789"),
    ("John", "987-65-4321")
], ["name", "ssn"])
df.show()

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 17, Finished, Available, Finished)

+-------------+-----------+
|         name|        ssn|
+-------------+-----------+
|RamReddy Myla|123-45-6789|
|         John|987-65-4321|
+-------------+-----------+



## 🔐 Method 1: Using `cryptography` Library

In [2]:
# Install cryptography
#%pip install cryptography

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 4, Finished, Available, Finished)

In [3]:
from cryptography.fernet import Fernet
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Generate key (store this securely in real use)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# UDFs for encryption and decryption
encrypt_udf = udf(lambda x: cipher_suite.encrypt(x.encode()).decode() if x else None, StringType())
decrypt_udf = udf(lambda x: cipher_suite.decrypt(x.encode()).decode() if x else None, StringType())

# Apply encryption
df_crypt = df.withColumn("ssn_encrypted", encrypt_udf("ssn"))
df_crypt = df_crypt.withColumn("ssn_decrypted", decrypt_udf("ssn_encrypted"))
df_crypt.show(truncate=False)

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 5, Finished, Available, Finished)

+-------------+-----------+----------------------------------------------------------------------------------------------------+-------------+
|name         |ssn        |ssn_encrypted                                                                                       |ssn_decrypted|
+-------------+-----------+----------------------------------------------------------------------------------------------------+-------------+
|RamReddy Myla|123-45-6789|gAAAAABoTwyILK3CxP0W30zT9MsRqWSDUaOS5wL2Tw1hoO6lIXnfa2H5I6VpX_VWjJgacCiFkRutx2ruG1fsUf3OLnjvvBqCmA==|123-45-6789  |
|John         |987-65-4321|gAAAAABoTwyIWbL9z87W8Y4IW__-_AnqqPagagR31qkg5bG1dXT84Y5b-vE4r5ssfmLOfcvTZEiRo_Fzfx_FobwUKlrO0OoDOA==|987-65-4321  |
+-------------+-----------+----------------------------------------------------------------------------------------------------+-------------+



## 🔐 Method 2: Using `pycryptodome` Library

In [6]:
# Install pycryptodome
%pip install pycryptodome

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 13, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.5 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [7]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64
import hashlib

# AES Utility Functions
BLOCK_SIZE = 16
PAD = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
UNPAD = lambda s: s[:-ord(s[len(s)-1:])]

def get_aes_key(secret):
    return hashlib.sha256(secret.encode()).digest()

def encrypt_aes(raw, key):
    raw = PAD(raw)
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(raw.encode())
    return base64.b64encode(iv + encrypted).decode()

def decrypt_aes(enc, key):
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(enc[16:])
    return UNPAD(decrypted.decode())

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 15, Finished, Available, Finished)

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
# Create UDFs for pycryptodome-based AES
passphrase = "MySuperSecretKey!"
key = get_aes_key(passphrase)

encrypt_udf = udf(lambda x: encrypt_aes(x, key) if x else None, StringType())
decrypt_udf = udf(lambda x: decrypt_aes(x, key) if x else None, StringType())

# Apply to DataFrame
df_pycrypto = df.withColumn("ssn_encrypted", encrypt_udf("ssn"))
df_pycrypto = df_pycrypto.withColumn("ssn_decrypted", decrypt_udf("ssn_encrypted"))
df_pycrypto.show(truncate=False)

StatementMeta(, c8555c92-db9b-4b29-999c-709fbcd5bec0, 18, Finished, Available, Finished)

+-------------+-----------+--------------------------------------------+-------------+
|name         |ssn        |ssn_encrypted                               |ssn_decrypted|
+-------------+-----------+--------------------------------------------+-------------+
|RamReddy Myla|123-45-6789|4nZVpTySNbMU5BSDh65DEhOhqBQ8juCvuwccPS4XX6E=|123-45-6789  |
|John         |987-65-4321|gHBE12809cyX4/ePHzP+w8Q1ANCUOfg1NPfKCJWhJXk=|987-65-4321  |
+-------------+-----------+--------------------------------------------+-------------+



## ✅ Summary
Both `cryptography` and `pycryptodome` allow secure encryption of sensitive fields in PySpark.

- Use **`cryptography`** for simplicity and strong security defaults.
- Use **`pycryptodome`** for full control over key, IV, and encryption modes.

In production, always store encryption keys securely using tools like **Azure Key Vault**.